In [1]:
# ASSOCIATION RULE LEARNING (BİRLİKTELİK KURALI ÖĞRENİMİ)
# 1. Veri Ön İşleme
# 2. ARL Veri Yapısını Hazırlama (Invoice-Product Matrix)
# 3. Birliktelik Kurallarının Çıkarılması
# 4. Çalışmanın Scriptini Hazırlama
# 5. Sepet Aşamasındaki Kullanıcılara Ürün Önerisinde Bulunma

<h1 style="font-size:18px;">VERİ ÖN İŞLEME</h1>

In [2]:


#!pip install mlxtend
import pandas as pd
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
# çıktının tek bir satırda olmasını sağlar.
pd.set_option('display.expand_frame_repr', False)
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
# https://archive.ics.uci.edu/ml/datasets/Online+Retail+II
#!pip install openpyxl

df_ = pd.read_excel("online_retail_II.xlsx",
                    sheet_name="Year 2010-2011")
#veri setini baştan yüklemek zaman aldığı için ilerde kullanmak üzere orjinal bir kopyasını alıyoruz
df = df_.copy()


/Users/mac/miniconda3/envs/tensorflow/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
import warnings
warnings.filterwarnings("ignore", message="`should_run_async` will not call `transform_cell` automatically in the future")


/Users/mac/miniconda3/envs/tensorflow/lib/python3.9/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
df.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [6]:
df.describe().T


,count,mean,min,25%,50%,75%,max,std
Quantity,541910.0,9.552234,-80995.0,1.0,3.0,10.0,80995.0,218.080957
InvoiceDate,541910,2011-07-04 13:35:22.342307584,2010-12-01 08:26:00,2011-03-28 11:34:00,2011-07-19 17:17:00,2011-10-19 11:27:00,2011-12-09 12:50:00,NaN
Price,541910.0,4.611138,-11062.06,1.25,2.08,4.13,38970.0,96.759765
Customer ID,406830.0,15287.68416,12346.0,13953.0,15152.0,16791.0,18287.0,1713.603074


-Veri setimizde problem var. Quantity ve price değişkenlerinde (-)eksi değerler var bunların olmaması lazım.
Bunun sebebi iade işlemlerinden dolayı ürünlerin geri dönüşerde (-) eksi işe işaretlenmiş olmasıdır.
-Çeyrekliklere baktığımızda da 25%, 50%, 75% normal aralıklarla artarken max değerin çok yüksek olması. 
Bunun sebebi outlayer değerler olabilir.

In [7]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [8]:
df.shape

(541910, 8)

Veri setindeki eksikleri, outlayer değerleri kırpacağız. (-)eksi ile başlayan quantity değerlerini de düzelteceğiz. Bu işlemler için bir fonksiyon yazıyoruz.


In [9]:
def retail_data_prep(dataframe):
    #boşlukları kaldır
    dataframe.dropna(inplace=True)
    #iadeleri kaldır(başında C harfi işe başlayanları)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    #(-)eksi değerleri kaldır
    dataframe = dataframe[dataframe["Quantity"] > 0]
    #(-)eksi değerleri kaldır
    dataframe = dataframe[dataframe["Price"] > 0]
    return dataframe

df = retail_data_prep(df)

In [10]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [11]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,397885.0,12.988208,1.0,2.0,6.0,12.0,80995.0,179.331551
InvoiceDate,397885,2011-07-10 23:41:56.419316992,2010-12-01 08:26:00,2011-04-07 11:12:00,2011-07-31 14:39:00,2011-10-20 14:33:00,2011-12-09 12:50:00,NaN
Price,397885.0,3.116525,0.001,1.25,1.95,3.75,8142.75,22.097861
Customer ID,397885.0,15294.416882,12346.0,13969.0,15159.0,16795.0,18287.0,1713.144421


Boşluklardan ve (-) eksi değerlerden kurtulmuş olduk. Şimdi outlayer değerlerle ilgileceğiz

Değişkenler için bir eşik değer hesaplayıp aykırı değerleri bu eşik değerle değiştireceğiz. Buna aynı zamanda baskılamada diyebiliriz. Öncelikle outlayer değerleri hesaplamak için bir fonksiyon yazıyoruz.

In [12]:
def outlier_thresholds(dataframe, variable):
    # veri setindeki seçtiğimiz değişken için 1% çeyrekliğini (kısmını) hesapla
    quartile1 = dataframe[variable].quantile(0.01)
    # veri setindeki seçtiğimiz değişken için 99% çeyrekliğini (kısmını) hesapla
    quartile3 = dataframe[variable].quantile(0.99)
    # Bu iki değer arasında bir range (dağılım) oluştur
    interquantile_range = quartile3 - quartile1
    # üst limit olarak 99% çeyreğin 1,5 katını al
    up_limit = quartile3 + 1.5 * interquantile_range
     # üst limit olarak 1% çeyreğin 1,5 katını al
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

Outlayer değerler için alt limit-üst limit belirledikten sonra bunları veri setimizde outlayer değerlerle değiştirmek için bir fonksiyon yazıyoruz.

In [13]:
def replace_with_thresholds(dataframe, variable):
    #daha önce yazdığımız fonksiyonla veri setimizde seçtiğimiz değişken için alt limit- üst limit hesapla
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    #veri setinin için alt limitten küçük olanları bul, onlari hesapladığımız alt limitle değiştir
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
     #veri setinin için üst limitten büyük olanları bul, onlari hesapladığımız üst limitle değiştir
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit




İleride tekrardan hesaplamak istersek diye boşlukları, c ile başlayan iadeleri ve (-)eksili değerleri kaldırması için baştaki yaptığımız ön işlemleride fonksiyona ekleyerek veri setimizdeki quantity ve price değişkenindeki outlayer değerler için   alt sınır ve üst sınır olarak hesaplayıp bunları değiştiren fonksiyonu tek bir fonksiyon olarak birleştirmiş olduk

In [14]:
def retail_data_prep(dataframe):
    #dataframe.dropna(inplace=True)
    #dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    #dataframe = dataframe[dataframe["Quantity"] > 0]
    #dataframe = dataframe[dataframe["Price"] > 0]

    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    return dataframe



In [15]:
df = retail_data_prep(df)

In [16]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [17]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,397885.0,11.83077,1.0,2.0,6.0,12.0,298.5,25.523052
InvoiceDate,397885,2011-07-10 23:41:56.419316992,2010-12-01 08:26:00,2011-04-07 11:12:00,2011-07-31 14:39:00,2011-10-20 14:33:00,2011-12-09 12:50:00,NaN
Price,397885.0,2.893492,0.001,1.25,1.95,3.75,37.06,3.227175
Customer ID,397885.0,15294.416882,12346.0,13969.0,15159.0,16795.0,18287.0,1713.144421


Yaptığımız tüm önişlemlerden sonra veri setimizdeki sorunları çözmüş olduk.

<h1 style="font-size:18px;">ARL VERİ YAPISINI HAZIRLAMAK</h1>

In [18]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6.0,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8.0,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6.0,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Veri setindeli invoice değişkenini sepetmiş gibi düşünüp ona göre dönüştürme yapacağız. Yanı satırda invoice sütünlarda ürünler olacak ve o faturada olup olmamasına göre 0 veya 1 değerini alacak. (matrise dönüştüreceğiz).

In [19]:
#ülke seçtik
df_fr = df[df['Country'] == "France"]

In [20]:
df_fr.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
26,536370,22728,ALARM CLOCK BAKELIKE PINK,24.0,2010-12-01 08:45:00,3.75,12583.0,France
27,536370,22727,ALARM CLOCK BAKELIKE RED,24.0,2010-12-01 08:45:00,3.75,12583.0,France
28,536370,22726,ALARM CLOCK BAKELIKE GREEN,12.0,2010-12-01 08:45:00,3.75,12583.0,France
29,536370,21724,PANDA AND BUNNIES STICKER SHEET,12.0,2010-12-01 08:45:00,0.85,12583.0,France
30,536370,21883,STARS GIFT TAPE,24.0,2010-12-01 08:45:00,0.65,12583.0,France


In [21]:
df_fr.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,8342.0,13.248202,1.0,6.0,10.0,12.0,298.5,18.031651
InvoiceDate,8342,2011-07-12 04:25:41.486454016,2010-12-01 08:45:00,2011-04-04 11:18:00,2011-08-15 12:53:00,2011-10-13 08:42:00,2011-12-09 12:50:00,NaN
Price,8342.0,3.232224,0.06,1.25,1.79,3.75,37.06,4.039973
Customer ID,8342.0,12678.406257,12413.0,12571.0,12678.0,12689.0,14277.0,277.266463


In [22]:
#Fatura ve ürün kırılımında miktarları gruplandırıyoruz. Yani bir faturadaki tüm ürünleri görmüş olduk. 
df_fr.groupby(['Invoice', 'Description']).agg({"Quantity": "sum"}).head(20).iloc[0:5, 0:5]


Quantity
Invoice Description                               
536370   SET 2 TEA TOWELS I LOVE LONDON       24.0
        ALARM CLOCK BAKELIKE GREEN            12.0
        ALARM CLOCK BAKELIKE PINK             24.0
        ALARM CLOCK BAKELIKE RED              24.0
        CHARLOTTE BAG DOLLY GIRL DESIGN       20.0

In [23]:
#ürünleri sütunlara yerleştiriyoruz. (unstack fonksiyonu kullanıyoruz) 
df_fr.groupby(['Invoice', 'Description']).agg({"Quantity": "sum"}).unstack()


Quantity                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [24]:
#boşlukları 0 ile dolduruyoruz. (fillna (0)fonksiyonuyla)
df_fr.groupby(['Invoice', 'Description']).agg({"Quantity": "sum"}).unstack().fillna(0).iloc[0:5, 0:5]

Quantity                                                                                                          
Description  50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER  I LOVE LONDON MINI BACKPACK  NINE DRAWER OFFICE TIDY  SET 2 TEA TOWELS I LOVE LONDON 
Invoice                                                                                                                                             
536370                                 0.0                0.0                          0.0                      0.0                             24.0
536852                                 0.0                0.0                          0.0                      0.0                              0.0
536974                                 0.0                0.0                          0.0                      0.0                              0.0
537065                                 0.0                0.0                          0.0                      0.0                              0.0
537463                                 0.0                0.0                          0.0                      0.0                              0.0

In [25]:
df_fr.groupby(['Invoice', 'Description']). \
    agg({"Quantity": "sum"}). \
    unstack(). \
    fillna(0). \
applymap(lambda x: 1 if x > 0 else 0).iloc[0:5, 0:5]

Quantity                                                                                                          
Description  50'S CHRISTMAS GIFT BAG LARGE  DOLLY GIRL BEAKER  I LOVE LONDON MINI BACKPACK  NINE DRAWER OFFICE TIDY  SET 2 TEA TOWELS I LOVE LONDON 
Invoice                                                                                                                                             
536370                                   0                  0                            0                        0                                1
536852                                   0                  0                            0                        0                                0
536974                                   0                  0                            0                        0                                0
537065                                   0                  0                            0                        0                                0
537463                                   0                  0                            0                        0                                0

Şu anki haliyle oluşturduğumuz matrisde sütünlarda ürün adları yer alıyor. Ama günlük hayata baktığımızda bunlar id lerine göre yerleştirilir. Bu yerleştireme matrise baktığımızda okunabilirliği biraz düşürür. Bunun önüne geçmek için istersek ürün adına göre istersek de stok koduna göre matrisi oluşturmak için fonksiyon yazıyoruz.

In [26]:
#Argüman olarak id veiyoruz. Ve default değeri false olarak ön tanımlıyoruz. Yani fonksiyonu direkt çağırırsa ürün
#adlarına göre gelecek. id=true olarak değiştirirsek stok kodlarına gelmiş olacak.
def create_invoice_product_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['Invoice', "StockCode"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)



In [27]:
fr_inv_pro_df = create_invoice_product_df(df_fr)
fr_inv_pro_df

Description   50'S CHRISTMAS GIFT BAG LARGE   DOLLY GIRL BEAKER   I LOVE LONDON MINI BACKPACK   NINE DRAWER OFFICE TIDY   SET 2 TEA TOWELS I LOVE LONDON    SPACEBOY BABY GIFT SET   TRELLIS COAT RACK  10 COLOUR SPACEBOY PEN  12 COLOURED PARTY BALLOONS  12 EGG HOUSE PAINTED WOOD  12 MESSAGE CARDS WITH ENVELOPES  12 PENCIL SMALL TUBE WOODLAND  12 PENCILS SMALL TUBE RED RETROSPOT  12 PENCILS SMALL TUBE SKULL  12 PENCILS TALL TUBE POSY  12 PENCILS TALL TUBE RED RETROSPOT  12 PENCILS TALL TUBE WOODLAND  15CM CHRISTMAS GLASS BALL 20 LIGHTS  16 PIECE CUTLERY SET PANTRY DESIGN  18PC WOODEN CUTLERY SET DISPOSABLE  20 DOLLY PEGS RETROSPOT  200 RED + WHITE BENDY STRAWS  3 HOOK HANGER MAGIC GARDEN  3 PIECE SPACEBOY COOKIE CUTTER SET  3 RAFFIA RIBBONS 50'S CHRISTMAS   3 STRIPEY MICE FELTCRAFT  3 TIER CAKE TIN RED AND CREAM  3 TRADITIONAl BISCUIT CUTTERS  SET  36 DOILIES DOLLY GIRL  36 DOILIES VINTAGE CHRISTMAS  36 FOIL HEART CAKE CASES  36 FOIL STAR CAKE CASES   36 PENCILS TUBE RED RETROSPOT  36 PENCILS TUBE SKULLS  36 PENCILS TUBE WOODLAND  3D HEARTS  HONEYCOMB PAPER GARLAND  3D TRADITIONAL CHRISTMAS STICKERS  3D VINTAGE CHRISTMAS STICKERS   4 IVORY DINNER CANDLES SILVER FLOCK  4 PINK DINNER CANDLE SILVER FLOCK  4 TRADITIONAL SPINNING TOPS  5 HOOK HANGER MAGIC TOADSTOOL  5 HOOK HANGER RED MAGIC TOADSTOOL  6 GIFT TAGS 50'S CHRISTMAS   6 GIFT TAGS VINTAGE CHRISTMAS   6 RIBBONS EMPIRE    6 RIBBONS RUSTIC CHARM  6 ROCKET BALLOONS   60 CAKE CASES DOLLY GIRL DESIGN  60 CAKE CASES VINTAGE CHRISTMAS  60 TEATIME FAIRY CAKE CASES  6PC WOOD PLATE SET DISPOSABLE  72 SWEETHEART FAIRY CAKE CASES  A PRETTY THANK YOU CARD  ABC TREASURE BOOK BOX   ADULT APRON APPLE DELIGHT  ADVENT CALENDAR GINGHAM SACK  AGED GLASS SILVER T-LIGHT HOLDER  AIRLINE BAG VINTAGE JET SET BROWN  AIRLINE BAG VINTAGE JET SET RED  AIRLINE BAG VINTAGE JET SET WHITE  AIRLINE BAG VINTAGE TOKYO 78  AIRLINE BAG VINTAGE WORLD CHAMPION   AIRLINE LOUNGE,METAL SIGN  ALARM CLOCK BAKELIKE CHOCOLATE  ALARM CLOCK BAKELIKE GREEN  ALARM CLOCK BAKELIKE IVORY  ALARM CLOCK BAKELIKE ORANGE  ALARM CLOCK BAKELIKE PINK  ALARM CLOCK BAKELIKE RED   ALPHABET HEARTS STICKER SHEET  ALUMINIUM STAMPED HEART  AMETHYST CHUNKY BEAD BRACELET W STR  ANGEL DECORATION STARS ON DRESS  ANTIQUE ALL GLASS CANDLESTICK  ANTIQUE GLASS DRESSING TABLE POT  ANTIQUE GLASS PEDESTAL BOWL  ANTIQUE SILVER BAUBLE LAMP    ANTIQUE SILVER T-LIGHT GLASS  ANTIQUE SILVER TEA GLASS ENGRAVED  APPLE BATH SPONGE  AREA PATROLLED METAL SIGN  ASS COLOUR GLOWING TIARAS  ASS FLORAL PRINT MULTI SCREWDRIVER  ASSORTED BOTTLE TOP  MAGNETS   ASSORTED COLOUR BIRD ORNAMENT  ASSORTED COLOUR MINI CASES  ASSORTED COLOUR T-LIGHT HOLDER  ASSORTED COLOURS SILK FAN  ASSORTED EASTER DECORATIONS  BELLS  ASSORTED EASTER GIFT TAGS  ASSORTED FLOWER COLOUR "LEIS"  ASSORTED TUTTI FRUTTI BRACELET  ASSORTED TUTTI FRUTTI MIRROR  ASSORTED TUTTI FRUTTI SMALL PURSE  ASSTD DESIGN 3D PAPER STICKERS  ASSTD DESIGN RACING CAR PEN  ASSTD FRUIT+FLOWERS FRIDGE MAGNETS  ASSTD RASTA KEY-CHAINS  BABUSHKA LIGHTS STRING OF 10  BAG 125g SWIRLY MARBLES  BAG 250g SWIRLY MARBLES  BAKING MOULD CHOCOLATE CUPCAKES  BAKING MOULD EASTER EGG MILK CHOC  BAKING MOULD EASTER EGG WHITE CHOC  BAKING MOULD HEART MILK CHOCOLATE  BAKING MOULD HEART WHITE CHOCOLATE  BAKING SET 9 PIECE RETROSPOT   BAKING SET SPACEBOY DESIGN  BALLOON ART MAKE YOUR OWN FLOWERS  BALLOON PUMP WITH 10 BALLOONS  BALLOON WATER BOMB PACK OF 35  BALLOONS  WRITING SET   BANQUET BIRTHDAY  CARD    BASKET OF TOADSTOOLS  BATH BUILDING BLOCK WORD  BEADED CRYSTAL HEART BLUE  LARGE  BELLE JARDINIERE CUSHION COVER  BEWARE OF THE CAT METAL SIGN   BIG DOUGHNUT FRIDGE MAGNETS  BILI NUT AND WOOD NECKLACE  BINGO SET  BIRD DECORATION GREEN POLKADOT  BIRD DECORATION RED RETROSPOT  BIRD HOUSE HOT WATER BOTTLE  BIRDS MOBILE VINTAGE DESIGN  BIRTHDAY CARD, RETRO SPOT  BISCUIT TIN 50'S CHRISTMAS  BISCUIT TIN VINTAGE CHRISTMAS  BISCUIT TIN VINTAGE GREEN  BISCUIT TIN VINTAGE RED  BLACK AND WHITE CAT BOWL  BLACK CANDELABRA T-LIGHT HOLDER  BLACK CHRISTMAS TR

In [28]:
fr_inv_pro_df = create_invoice_product_df(df_fr, id=True)
fr_inv_pro_df

StockCode  10002  10120  10125  10135  11001  15036  15039  16012  16048  16218  16219  16225  16236  16237  16238  17174  20615  20617  20658  20665  20668  20674  20675  20676  20677  20679  20681  20682  20684  20685  20686  20702  20704  20711  20712  20713  20717  20718  20719  20723  20724  20725  20726  20727  20728  20749  20750  20751  20767  20777  20819  20832  20866  20914  20961  20963  20966  20967  20971  20972  20973  20974  20975  20977  20978  20979  20981  20983  20984  20992  20996  21012  21026  21027  21030  21034  21035  21039  21042  21054  21055  21056  21058  21059  21061  21062  21063  21064  21065  21068  21069  21070  21078  21080  21084  21086  21087  21088  21090  21094  21095  21096  21098  21108  21110  21111  21114  21116  21121  21122  21123  21124  21125  21126  21135  21136  21137  21154  21155  21156  21161  21162  21163  21164  21165  21166  21174  21175  21181  21188  21194  21195  21196  21197  21198  21199  21200  21201  21202  21204  21205  21206  21207  21208  21209  21210  21212  21213  21216  21217  21218  21219  21221  21224  21231  21232  21238  21239  21240  21242  21243  21244  21245  21246  21248  21249  21251  21257  21258  21259  21262  21284  21286  21294  21306  21326  21328  21329  21340  21351  21354  21355  21356  21358  21361  21365  21369  21371  21372  21374  21375  21377  21380  21381  21383  21385  21394  21395  21398  21399  21402  21403  21407  21408  21411  21424  21425  21426  21427  21428  21429  21430  21439  21442  21452  21463  21466  21467  21469  21470  21471  21472  21479  21481  21484  21485  21494  21495  21497  21498  21499  21500  21506  21507  21508  21509  21519  21523  21524  21527  21528  21531  21533  21535  21537  21539  21544  21555  21556  21557  21558  21559  21561  21563  21564  21576  21577  21578  21579  21581  21584  21591  21621  21622  21623  21624  21643  21648  21650  21658  21668  21669  21671  21672  21673  21675  21677  21678  21679  21680  21693  21696  21698  21700  21703  21704  21706  21707  21708  21709  21710  21711  21715  21716  21719  21721  21723  21724  21725  21731  21733  21739  21746  21747  21749  21754  21755  21756  21770  21774  21786  21787  21790  21791  21812  21813  21818  21819  21821  21828  21829  21832  21833  21843  21844  21845  21864  21865  21866  21871  21872  21875  21877  21880  21881  21882  21883  21884  21888  21889  21890  21891  21892  21894  21900  21901  21905  21906  21907  21908  21912  21913  21914  21915  21916  21917  21918  21922  21926  21928  21929  21930  21931  21932  21933  21934  21935  21936  21937  21943  21946  21947  21948  21949  21955  21967  21972  21974  21975  21976  21977  21980  21981  21982  21983  21985  21986  21987  21988  21989  21990  22023  22024  22025  22026  22027  22028  22029  22032  22035  22037  22041  22043  22044  22045  22046  22051  22052  22059  22063  22064  22065  22068  22070  22071  22072  22073  22076  22077  22078  22079  22080  22081  22082  22083  22084  22085  22086  22087  22088  22089  22090  22091  22093  22094  22099  22109  22110  22111  22112  22113  22114  22115  22116  22118  22120  22121  22124  22129  22130  22131  22132  22133  22134  22136  22138  22139  22141  22142  22144  22149  22150  22151  22153  22170  22174  22175  22176  22178  22179  22180  22181  22189  22191  22192  22193  22195  22197  22198  22199  22200  22203  22204  22207  22208  22209  22210  22211  22215  22219  22221  22222  22223  22224  22228  22231  22232  22236  22242  22243  22244  22254  22255  22262  22264  22269  22271  22272  22273  22274  22282  22285  22294  22295  22299  22300  22301  22302  22303  22305  22306  22309  22311  22312  22313  22314  22315  22318  22319  22320  22321  22322  22324  22325  22326  22327  22328  22329  22331  22332  22333  22334  22339  22340  22341  22343  22344  22346  22348  22349  22350  22352  22354  22355  22356  22357  22358  22361  22362  22365  22366  22367  22371  22372  22374  22375  22376  22377  22378  22379 

Daha sonra analiz yaparken hangi ID nin hangi ürüne ait olduğunu hızlı bir dönüşümle görmek istersek diye bunun için çok kısa bir fonksiyon yazıyoruz.

In [29]:
#veri seti içinde gez stock code karşılık gelen description getir
def check_id(dataframe, stock_code):
    product_name = dataframe[dataframe["StockCode"] == stock_code][["Description"]].values[0].tolist()
    print(product_name)

In [30]:
check_id(df_fr, 15036)

['ASSORTED COLOURS SILK FAN']


<h1 style="font-size:18px;">BİRLİKTELİK KURALLARININ ÇIKARILMASI</h1>

In [31]:
frequent_itemsets = apriori(fr_inv_pro_df,
                            min_support=0.01,
                            use_colnames=True)

/Users/mac/miniconda3/envs/tensorflow/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [32]:
frequent_itemsets.sort_values("support", ascending=False)

,support,itemsets
538,0.773779,(POST)
387,0.187661,(23084)
107,0.179949,(21731)
243,0.172237,(22554)
245,0.169666,(22556)
...,...,...
18793,0.010283,"(22729, 21086, 22326, 22551)"
18787,0.010283,"(23256, 21086, 22492, 22326)"
18786,0.010283,"(22728, 21086, 22492, 22326)"
18785,0.010283,"(21086, 22492, 22326, 22727)"


Her bir ürünün olasılığını vermiş oldu. Bunun üzerinden birliktelik kurallarını çıkartacağız.

In [33]:
rules = association_rules(frequent_itemsets, metric="support",min_threshold=0.01)                       

In [34]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(10002),(21791),0.020566,0.028278,0.010283,0.500000,17.681818,0.009701,1.943445,0.963255
1,(21791),(10002),0.028278,0.020566,0.010283,0.363636,17.681818,0.009701,1.539111,0.970899
2,(10002),(21915),0.020566,0.069409,0.010283,0.500000,7.203704,0.008855,1.861183,0.879265
3,(21915),(10002),0.069409,0.020566,0.010283,0.148148,7.203704,0.008855,1.149771,0.925414
4,(10002),(22551),0.020566,0.136247,0.010283,0.500000,3.669811,0.007481,1.727506,0.742782


antecedents ilk ürün, consequents ikinci ürün, antecedent support ilk ürünün görülme olasılığı, consequent support ikinci ürünün görülme olasılığı, support iki ürünğn birlikte görülme olasılığı, confidence ilk ürün satın alındığında ikinci ürünün de satın alınma olasılığı, lift ilk ürünün satın alınmasının ikinci ürünün satın alınma olasılığını kaç kat arttırdığı

In [35]:
rules[(rules["support"]>0.05) & (rules["confidence"]>0.1) & (rules["lift"]>5)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1606,(21080),(21086),0.133676,0.138817,0.102828,0.769231,5.541311,0.084271,3.731791,0.945994
1607,(21086),(21080),0.138817,0.133676,0.102828,0.740741,5.541311,0.084271,3.341535,0.951642
1608,(21080),(21094),0.133676,0.128535,0.102828,0.769231,5.984615,0.085646,3.776350,0.961424
1609,(21094),(21080),0.128535,0.133676,0.102828,0.800000,5.984615,0.085646,4.331620,0.955752
1776,(21086),(21094),0.138817,0.128535,0.123393,0.888889,6.915556,0.105550,7.843188,0.993284
...,...,...,...,...,...,...,...,...,...,...
213940,"(22727, POST)","(22728, 22726)",0.089974,0.074550,0.059126,0.657143,8.814778,0.052418,2.699229,0.974208
213941,"(22726, 22727)","(22728, POST)",0.079692,0.092545,0.059126,0.741935,8.017025,0.051751,3.516388,0.951057
213942,(22728),"(22727, 22726, POST)",0.102828,0.074550,0.059126,0.575000,7.712931,0.051460,2.177529,0.970101
213944,(22726),"(22728, 22727, POST)",0.097686,0.069409,0.059126,0.605263,8.720273,0.052346,2.357498,0.981172


In [36]:
check_id(df_fr, 21086)

['SET/6 RED SPOTTY PAPER CUPS']


In [37]:
#confidance göre sıralama yapıyoruz
rules[(rules["support"]>0.05) & (rules["confidence"]>0.1) & (rules["lift"]>5)]. \
sort_values("confidence", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
23707,"(21080, 21094)",(21086),0.102828,0.138817,0.100257,0.975000,7.023611,0.085983,34.447301,0.955918
23706,"(21080, 21086)",(21094),0.102828,0.128535,0.100257,0.975000,7.585500,0.087040,34.858612,0.967673
108821,"(21080, 21086, POST)",(21094),0.084833,0.128535,0.082262,0.969697,7.544242,0.071358,28.758355,0.947858
108822,"(21080, 21094, POST)",(21086),0.084833,0.138817,0.082262,0.969697,6.985410,0.070486,28.419023,0.936271
1777,(21094),(21086),0.128535,0.138817,0.123393,0.960000,6.915556,0.105550,21.529563,0.981563
...,...,...,...,...,...,...,...,...,...,...
7212,(22629),(22630),0.125964,0.100257,0.071979,0.571429,5.699634,0.059351,2.099400,0.943382
62248,(22630),"(22629, POST)",0.100257,0.100257,0.053985,0.538462,5.370809,0.043933,1.949443,0.904490
62245,"(22629, POST)",(22630),0.100257,0.100257,0.053985,0.538462,5.370809,0.043933,1.949443,0.904490
62247,(22629),"(22630, POST)",0.125964,0.074550,0.053985,0.428571,5.748768,0.044594,1.619537,0.945098


(21080, 21094) bu iki ürün birlikte alındığında support değeri düşükmüş gozüksede bu ikin ürün alındıktan sonra 21086 nolu ürünün birlikte alınma olasılığı 0,97 yani neredeyse 100% 

<h1 style="font-size:18px;">ÇALIŞMA SCRİPTİNİ HAZIRLAMA</h1>

Tüm yazdığımız fonksiyonları tek bir fonksiyonda topluyoruz

In [38]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

def retail_data_prep(dataframe):
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[dataframe["Quantity"] > 0]
    dataframe = dataframe[dataframe["Price"] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    return dataframe


def create_invoice_product_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['Invoice', "StockCode"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)


def check_id(dataframe, stock_code):
    product_name = dataframe[dataframe["StockCode"] == stock_code][["Description"]].values[0].tolist()
    print(product_name)


def create_rules(dataframe, id=True, country="France"):
    dataframe = dataframe[dataframe['Country'] == country]
    dataframe = create_invoice_product_df(dataframe, id)
    frequent_itemsets = apriori(dataframe, min_support=0.01, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
    return rules


In [39]:
#tekrardan orjinal veri setini çağırıyoruz
df = df_.copy()

In [40]:
df = retail_data_prep(df)
rules = create_rules(df)

/Users/mac/miniconda3/envs/tensorflow/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [41]:
rules[(rules["support"]>0.05) & (rules["confidence"]>0.1) & (rules["lift"]>5)]. \
sort_values("confidence", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
23707,"(21080, 21094)",(21086),0.102828,0.138817,0.100257,0.975000,7.023611,0.085983,34.447301,0.955918
23706,"(21080, 21086)",(21094),0.102828,0.128535,0.100257,0.975000,7.585500,0.087040,34.858612,0.967673
108821,"(21080, 21086, POST)",(21094),0.084833,0.128535,0.082262,0.969697,7.544242,0.071358,28.758355,0.947858
108822,"(21080, 21094, POST)",(21086),0.084833,0.138817,0.082262,0.969697,6.985410,0.070486,28.419023,0.936271
1777,(21094),(21086),0.128535,0.138817,0.123393,0.960000,6.915556,0.105550,21.529563,0.981563
...,...,...,...,...,...,...,...,...,...,...
7212,(22629),(22630),0.125964,0.100257,0.071979,0.571429,5.699634,0.059351,2.099400,0.943382
62248,(22630),"(22629, POST)",0.100257,0.100257,0.053985,0.538462,5.370809,0.043933,1.949443,0.904490
62245,"(22629, POST)",(22630),0.100257,0.100257,0.053985,0.538462,5.370809,0.043933,1.949443,0.904490
62247,(22629),"(22630, POST)",0.125964,0.074550,0.053985,0.428571,5.748768,0.044594,1.619537,0.945098


<h1 style="font-size:18px;">KULLANICILARA ÜRÜN ÖNERİSİNDE BULUNMA</h1>


Kullanıcı online mağazada gezerken bir ürünü sepete ekledi

In [44]:
# Örnek:
# Kullanıcı örnek ürün id: 22492

In [45]:
product_id = 22492
check_id(df, product_id)

['MINI PAINT SET VINTAGE ']


Kullanıcıya ne önereceğiz??

In [50]:
#lifte göre sıraladık
sorted_rules = rules.sort_values("lift", ascending=False)


In [51]:
recommendation_list = [] #boş bir liste oluştur

for i, product in enumerate(sorted_rules["antecedents"]): #ilk ürün sütununda gez 
    for j in list(product): #bazı satırlar ikili forma olduğu için bunu bir listeye çevir
        if j == product_id: #verdiğimiz ürün numarasına bulursan
            recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0]) 
            # bulduğun satıra karşılık gelen ikinci ürün sütunundak ürünü getir



In [52]:
# ilk 3 gözlemi getir
recommendation_list[0:3]

check_id(df, 22326)

['ROUND SNACK BOXES SET OF4 WOODLAND ']


In [64]:

def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

    return recommendation_list[0:rec_count]

In [65]:
arl_recommender(rules, 22492, 1)

[22556]

In [66]:
arl_recommender(rules, 22492, 2)

[22556, 22551]

In [77]:
arl_recommender(rules, 22492, 3)

[22556, 22551, 22326]

In [78]:
def get_recommendations(rules_df, product_id, rec_count=1):
    recommended_ids = arl_recommender(rules_df, product_id, rec_count)
    recommended_names = []
    for product in recommended_ids:
        name = check_id(df, product)
        recommended_names.append(name)
        
    return recommended_names


In [80]:
# Örnek kullanım
get_recommendations(rules, 22492, rec_count=2)

['PLASTERS IN TIN CIRCUS PARADE ']
['PLASTERS IN TIN SPACEBOY']


[None, None]